In [ ]:
library(readxl)
library(dplyr)
library('pwr')
library(plotly)
library(reshape2)
library(ggplot2)
library(car)

In [ ]:
Victoria_primaria <- read_excel("C:/Users/ayrto/OneDrive/Escritorio/Exploración de Datos Multivariados/1 - Trabajo Practico/VICTORIA_BASE_PRIMARIA_LIMPIA.xlsx")

### Actividad 1 (cambiar planteamiento)

In [ ]:
# Segmentamos segun el ciclo
ciclo_basico <- filter(Victoria_primaria, Año_curso == "PRIMERO" | Año_curso == "SEGUNDO" | Año_curso == "TERCERO")$Promedio
ciclo_orientado <- filter(Victoria_primaria, Año_curso == "CUARTO" | Año_curso == "QUINTO" | Año_curso == "SEXTO")$Promedio

# Calculamos normalidad de los datos
ks.test(ciclo_basico, "pnorm") #p-value < 2.2e-16
ks.test(ciclo_orientado, "pnorm") #p-value < 2.2e-16

# Medianas de los datos
median(ciclo_basico)
median(ciclo_orientado)

# H0 ---> Los promedios de ciclo basico son iguales a los de ciclo orientado
# H1 ---> Los promedios de ciclo basico difieren de los promedios de ciclo orientado
wilcox.test(ciclo_basico, ciclo_orientado, alternative = "two.sided", conf.level = 0.90)

# La evidencia es suficiente para decir que los promedios entre el ciclo basico y el orientado
#difieren significativamente



### Actividad 2 (cambiar planteamiento)

### A*** MEJORAR ANALISIS

In [ ]:
# Hacemos un data frame con los grados de Ciclo Basico
datos_cbc <- bind_rows(
  mutate(filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA"), Año_curso = "PRIMERO"),
  mutate(filter(Victoria_primaria, Año_curso == "SEGUNDO", asignatura == "MATEMATICA"), Año_curso = "SEGUNDO"),
  mutate(filter(Victoria_primaria, Año_curso == "TERCERO", asignatura == "MATEMATICA"), Año_curso = "TERCERO")
)

# Boxplot interactivo
plot_ly(
  data = datos_cbc,
  type = "box",
  x = ~Año_curso,
  y = ~Promedio,
  color = ~Año_curso, 
  colors = c("blue", "red", "orange")
)

In [ ]:
# Hacemos un data frame con los grados de Ciclo Orientado
datos_cbc <- bind_rows(
  mutate(filter(Victoria_primaria, Año_curso == "CUARTO"), Año_curso = "CUARTO"),
  mutate(filter(Victoria_primaria, Año_curso == "QUINTO"), Año_curso = "QUINTO"),
  mutate(filter(Victoria_primaria, Año_curso == "SEXTO"), Año_curso = "SEXTO")
)

# Boxplot interactivo
plot_ly(
  data = datos_cbc,
  type = "box",
  x = ~Año_curso,
  y = ~Promedio,
  color = ~Año_curso, 
  colors = c("skyblue", "green", "pink")
)

### B***

In [ ]:
# Segmentamos los datos de los grados del CBC
primero <- filter(Victoria_primaria, Año_curso == "PRIMERO")$Promedio
segundo <- filter(Victoria_primaria, Año_curso == "SEGUNDO")$Promedio
tercero <- filter(Victoria_primaria, Año_curso == "TERCERO")$Promedio

# Pasamos los promedios a tipo numerico
notas= as.numeric(c(primero, segundo, tercero))

# Segun la cantidad de datos en cada grado, se repite su nombre
curso <- as.factor(c(rep("PRIMERO", length(primero)), rep("SEGUNDO",length(segundo)), rep("TERCERO", length(tercero))))

# Armamos el data frame
notas_cbc <- data.frame(curso, notas)

# Mostramos la media de cada grado y su cantidad de datos
notas_cbc %>% group_by(curso) %>%
  summarise(mean = round(mean(notas),1), datos= n())

# Test de ANOVA
fm1 = aov( notas ~ curso )
#H0 ---> Los tres grados del CBC tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm1)

In [ ]:
# Segmentamos los datos de los grados del Ciclo Orientado
cuarto <- filter(Victoria_primaria, Año_curso == "CUARTO")$Promedio
quinto <- filter(Victoria_primaria, Año_curso == "QUINTO")$Promedio
sexto <- filter(Victoria_primaria, Año_curso == "SEXTO")$Promedio

# Pasamos los promedios a tipo numerico
notas= as.numeric(c(cuarto, quinto, sexto))

# Segun la cantidad de datos en cada grado, se repite su nombre
curso <- as.factor(c(rep("CUARTO", length(cuarto)), rep("QUINTO",length(quinto)), rep("SEXTO", length(sexto))))

# Armamos el data frame
notas_co <- data.frame(curso, notas)

# Mostramos la media de cada grado y su cantidad de datos
notas_co %>% group_by(curso) %>%
  summarise(mean = round(mean(notas),1), datos= n())

# Test de ANOVA
fm2 = aov( notas ~ curso )
#H0 ---> Los tres grados del Ciclo Orientado tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm2)

### C***

In [ ]:
# CBC ---> Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# CO ---> Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

### D***

In [ ]:
# Test HSD de Tukey para CBC
TukeyHSD(fm1, "curso", conf.level = .95)
# TERCERO-SEGUNDO ---> p adj = 0.0152416

# Test HSD de Tukey para CO
TukeyHSD(fm2, "curso", conf.level = .95)
# SEXTO-CUARTO ---> p adj = 0.0016141
# SEXTO-QUINTO ---> p adj = 0.0191035

### E***

In [ ]:
# Prueba de normalidad para los grados de CBC
ks.test(notas_cbc$notas[notas_cbc$curso == "PRIMERO"], "pnorm")
ks.test(notas_cbc$notas[notas_cbc$curso == "SEGUNDO"], "pnorm")
ks.test(notas_cbc$notas[notas_cbc$curso == "TERCERO"], "pnorm")

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para los grados de CO
ks.test(notas_co$notas[notas_co$curso == "CUARTO"], "pnorm")
ks.test(notas_co$notas[notas_co$curso == "QUINTO"], "pnorm")
ks.test(notas_co$notas[notas_co$curso == "SEXTO"], "pnorm")

# El test de ANOVA no se puede aplicar ya que los datos no siguen una distribución normal

### F***

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

print("----------------------------------------------------------------------------------------")

# Test no parametrico para CBC
#H0 ---> Los tres grados del CBC tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_cbc$notas, notas_cbc$curso)
pairwise.wilcox.test(notas_cbc$notas, notas_cbc$curso)

# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los grupos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para CO
#H0 ---> Los tres grados del CO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_co$notas, notas_co$curso)
pairwise.wilcox.test(notas_co$notas, notas_co$curso)

# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los grupos

### Actividad 3

### MATEMATICA

### A***

In [ ]:
# Creamos un dataframe combinado con los datos de todos los años de curso
datos_completos <- bind_rows(
  mutate(filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA"), Año_curso = "PRIMERO"),
  mutate(filter(Victoria_primaria, Año_curso == "SEGUNDO", asignatura == "MATEMATICA"), Año_curso = "SEGUNDO"),
  mutate(filter(Victoria_primaria, Año_curso == "TERCERO", asignatura == "MATEMATICA"), Año_curso = "TERCERO"),
  mutate(filter(Victoria_primaria, Año_curso == "CUARTO", asignatura == "MATEMATICA"), Año_curso = "CUARTO"),
  mutate(filter(Victoria_primaria, Año_curso == "QUINTO", asignatura == "MATEMATICA"), Año_curso = "QUINTO"),
  mutate(filter(Victoria_primaria, Año_curso == "SEXTO", asignatura == "MATEMATICA"), Año_curso = "SEXTO")
)

# Filtramos los datos para el turno MAÑANA
datos_mañana <- datos_completos %>%
  filter(turno == "MAÑANA")

# Filtramos los datos para el turno TARDE
datos_tarde <- datos_completos %>%
  filter(turno == "TARDE")

# Filtramos los datos para el turno COMPLETO
datos_completo <- datos_completos %>%
filter(turno == "COMPLETO")

# Creamos un gráfico interactivo
plot <- plot_ly() %>%
  add_boxplot(data = datos_mañana, x = ~Año_curso, y = ~Promedio, name = "MAÑANA", marker = list(color = "blue"), offsetgroup = 1) %>%
  add_boxplot(data = datos_tarde, x = ~Año_curso, y = ~Promedio, name = "TARDE", marker = list(color = "red"), offsetgroup = 2) %>%
  add_boxplot(data = datos_completo, x = ~Año_curso, y = ~Promedio, name = "COMPLETO", marker = list(color = "greem"), offsetgroup = 3) %>%
  layout(
    title = "Comparación de Promedios de Matemáticas por Año de Curso y Turno",
    xaxis = list(title = "Año de Curso"),
    yaxis = list(title = "Promedio"),
    barmode = "group",
    boxmode = "group"
  )
plot

### B***

In [ ]:
# Analisis para primer grado en MATEMATICAS
datos_mañana1 <- filter(Victoria_primaria, Año_curso == "PRIMERO" & turno == "MAÑANA" & asignatura == "MATEMATICA")$Promedio
datos_tarde1 <- filter(Victoria_primaria, Año_curso == "PRIMERO" & turno == "TARDE" & asignatura == "MATEMATICA")$Promedio
datos_completo1 <- filter(Victoria_primaria, Año_curso == "PRIMERO" & turno == "COMPLETO" & asignatura == "MATEMATICA")$Promedio

# Pasamos los promedios a tipo numerico
primero = as.numeric(c(datos_mañana1, datos_tarde1, datos_completo1))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana1)), rep("TARDE",length(datos_tarde1)), rep("COMPLETO", length(datos_completo1))))
# Armamos el data frame
notas_primero <- data.frame(turno, primero)

# Mostramos la media de cada turno y su cantidad de datos
notas_primero %>% group_by(turno) %>%
  summarise(Media = round(mean(primero),1), datos= n())

# Test de ANOVA
fm1 = aov( primero ~ turno )
#H0 ---> Los tres turnos de PRIMERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm1)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(75, 272, 217), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para segundo grado en MATEMATICA
datos_mañana2 <- filter(Victoria_primaria, Año_curso == "SEGUNDO" & turno == "MAÑANA" & asignatura == "MATEMATICA")$Promedio
datos_tarde2 <- filter(Victoria_primaria, Año_curso == "SEGUNDO" & turno == "TARDE" & asignatura == "MATEMATICA")$Promedio
datos_completo2 <- filter(Victoria_primaria, Año_curso == "SEGUNDO" & turno == "COMPLETO" & asignatura == "MATEMATICA")$Promedio

# Pasamos los promedios a tipo numerico
segundo = as.numeric(c(datos_mañana2, datos_tarde2, datos_completo2))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana2)), rep("TARDE",length(datos_tarde2)), rep("COMPLETO", length(datos_completo2))))
# Armamos el data frame
notas_segundo <- data.frame(turno, segundo)

# Mostramos la media de cada turno y su cantidad de datos
notas_segundo %>% group_by(turno) %>%
  summarise(Media = round(mean(segundo),1), datos= n())

# Test de ANOVA
fm2 = aov( segundo ~ turno )
#H0 ---> Los tres turnos de SEGUNDO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm2)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(83, 237, 237), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.94
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para tercer grado en MATEMATICA
datos_mañana3 <- filter(Victoria_primaria, Año_curso == "TERCERO" & turno == "MAÑANA" & asignatura == "MATEMATICA")$Promedio
datos_tarde3 <- filter(Victoria_primaria, Año_curso == "TERCERO" & turno == "TARDE" & asignatura == "MATEMATICA")$Promedio
datos_completo3 <- filter(Victoria_primaria, Año_curso == "TERCERO" & turno == "COMPLETO" & asignatura == "MATEMATICA")$Promedio

# Pasamos los promedios a tipo numerico
tercero = as.numeric(c(datos_mañana3, datos_tarde3, datos_completo3))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana3)), rep("TARDE",length(datos_tarde3)), rep("COMPLETO", length(datos_completo3))))
# Armamos el data frame
notas_tercero <- data.frame(turno, tercero)

# Mostramos la media de cada turno y su cantidad de datos
notas_tercero %>% group_by(turno) %>%
  summarise(Media = round(mean(tercero),1), datos= n())

# Test de ANOVA
fm3 = aov( tercero ~ turno )
#H0 ---> Los tres turnos de TERCERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm3)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(91, 256, 248), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.96
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para cuarto grado en MATEMATICA
datos_mañana4 <- filter(Victoria_primaria, Año_curso == "CUARTO" & turno == "MAÑANA" & asignatura == "MATEMATICA")$Promedio
datos_tarde4 <- filter(Victoria_primaria, Año_curso == "CUARTO" & turno == "TARDE" & asignatura == "MATEMATICA")$Promedio
datos_completo4 <- filter(Victoria_primaria, Año_curso == "CUARTO" & turno == "COMPLETO" & asignatura == "MATEMATICA")$Promedio

# Pasamos los promedios a tipo numerico
cuarto = as.numeric(c(datos_mañana4, datos_tarde4, datos_completo4))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana4)), rep("TARDE",length(datos_tarde4)), rep("COMPLETO", length(datos_completo4))))
# Armamos el data frame
notas_cuarto <- data.frame(turno, cuarto)

# Mostramos la media de cada turno y su cantidad de datos
notas_cuarto %>% group_by(turno) %>%
  summarise(mean = round(mean(cuarto),1), datos= n())

# Test de ANOVA
fm4 = aov( cuarto ~ turno )
#H0 ---> Los tres turnos de CUARTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm4)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(66, 240, 209), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.88
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para quinto grado en MATEMATICA
datos_mañana5 <- filter(Victoria_primaria, Año_curso == "QUINTO" & turno == "MAÑANA" & asignatura == "MATEMATICA")$Promedio
datos_tarde5 <- filter(Victoria_primaria, Año_curso == "QUINTO" & turno == "TARDE" & asignatura == "MATEMATICA")$Promedio
datos_completo5 <- filter(Victoria_primaria, Año_curso == "QUINTO" & turno == "COMPLETO" & asignatura == "MATEMATICA")$Promedio

# Pasamos los promedios a tipo numerico
quinto = as.numeric(c(datos_mañana5, datos_tarde5, datos_completo5))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana5)), rep("TARDE",length(datos_tarde5)), rep("COMPLETO", length(datos_completo5))))
# Armamos el data frame
notas_quinto <- data.frame(turno, quinto)

# Mostramos la media de cada turno y su cantidad de datos
notas_quinto %>% group_by(turno) %>%
  summarise(mean = round(mean(quinto),1), datos= n())

# Test de ANOVA
fm5 = aov( quinto ~ turno )
#H0 ---> Los tres turnos de QUINTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm5)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(76, 248, 216), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para sexto grado en MATEMATICA
datos_mañana6 <- filter(Victoria_primaria, Año_curso == "SEXTO" & turno == "MAÑANA" & asignatura == "MATEMATICA")$Promedio
datos_tarde6 <- filter(Victoria_primaria, Año_curso == "SEXTO" & turno == "TARDE" & asignatura == "MATEMATICA")$Promedio
datos_completo6 <- filter(Victoria_primaria, Año_curso == "SEXTO" & turno == "COMPLETO" & asignatura == "MATEMATICA")$Promedio

# Pasamos los promedios a tipo numerico
sexto = as.numeric(c(datos_mañana6, datos_tarde6, datos_completo6))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana6)), rep("TARDE",length(datos_tarde6)), rep("COMPLETO", length(datos_completo6))))
# Armamos el data frame
notas_sexto <- data.frame(turno, sexto)

# Mostramos la media de cada turno y su cantidad de datos
notas_sexto %>% group_by(turno) %>%
  summarise(mean = round(mean(sexto),1), datos= n())

# Test de ANOVA
fm6 = aov( sexto ~ turno )
#H0 ---> Los tres turnos de SEXTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm6)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(85, 262, 219), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.95
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

### C***

In [ ]:
# Despues de hacer el test de ANOVA para la variable turno de todos los grados de primaria, los resultados
#se repitieron en todos los cursos, al menos una media de un conjunto de los datos es distinta a las demas
#en algunos casos las diferencias eran muy significativas y en otras muy leves.

### D***

In [ ]:
# Test HSD de Tukey para PRIMERO
TukeyHSD(fm1, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0000013
# TARDE-COMPLETO -> 0.0000140

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(75, 272, 217), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para SEGUNDO
TukeyHSD(fm2, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0
# TARDE-COMPLETO -> 0.0

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(83, 237, 237), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.94
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para TERCERO
TukeyHSD(fm3, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0
# TARDE-COMPLETO -> 0.0

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(91, 256, 248), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.96
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para CUARTO
TukeyHSD(fm4, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0449610
# TARDE-COMPLETO -> 0.0148709

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(66, 240, 209), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.88
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para QUINTO
TukeyHSD(fm5, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0002655

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(76, 248, 216), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para SEXTO
TukeyHSD(fm6, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0
# TARDE-COMPLETO -> 0.0

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(85, 262, 219), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.95
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

### E***

In [ ]:
# Prueba de normalidad para PRIMERO
shapiro.test(notas_primero$primero[notas_primero$turno == "MAÑANA"])
shapiro.test(notas_primero$primero[notas_primero$turno == "TARDE"])
shapiro.test(notas_primero$primero[notas_primero$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para SEGUNDO
shapiro.test(notas_segundo$segundo[notas_segundo$turno == "MAÑANA"])
shapiro.test(notas_segundo$segundo[notas_segundo$turno == "TARDE"])
shapiro.test(notas_segundo$segundo[notas_segundo$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para TERCERO
shapiro.test(notas_tercero$tercero[notas_tercero$turno == "MAÑANA"])
shapiro.test(notas_tercero$tercero[notas_tercero$turno == "TARDE"])
shapiro.test(notas_tercero$tercero[notas_tercero$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para CUARTO
shapiro.test(notas_cuarto$cuarto[notas_cuarto$turno == "MAÑANA"])
shapiro.test(notas_cuarto$cuarto[notas_cuarto$turno == "TARDE"])
shapiro.test(notas_cuarto$cuarto[notas_cuarto$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para QUINTO
shapiro.test(notas_quinto$quinto[notas_quinto$turno == "MAÑANA"])
shapiro.test(notas_quinto$quinto[notas_quinto$turno == "TARDE"])
shapiro.test(notas_quinto$quinto[notas_quinto$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para SEXTO
shapiro.test(notas_sexto$sexto[notas_sexto$turno == "MAÑANA"])
shapiro.test(notas_sexto$sexto[notas_sexto$turno == "TARDE"])
shapiro.test(notas_sexto$sexto[notas_sexto$turno == "COMPLETO"])

# Ningun conjunto de datos cumple con los supuestos para realizar ANOVA, asi que el análisis realizado 
#anteriormente no es confiable

### F***

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

print("----------------------------------------------------------------------------------------")

# Test no parametrico para PRIMERO
#H0 ---> Los tres turnos de PRIMERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_primero$primero, notas_primero$turno)
pairwise.wilcox.test(notas_primero$primero, notas_primero$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre la 
#media de los promedios de los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para SEGUNDO
#H0 ---> Los tres turnos de SEGUNDO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_segundo$segundo, notas_segundo$turno)
pairwise.wilcox.test(notas_segundo$segundo, notas_segundo$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para TERCERO
#H0 ---> Los tres turnos de TERCERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_tercero$tercero, notas_tercero$turno)
pairwise.wilcox.test(notas_tercero$tercero, notas_tercero$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para CUARTO
#H0 ---> Los tres turnos de CUARTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_cuarto$cuarto, notas_cuarto$turno)
pairwise.wilcox.test(notas_cuarto$cuarto, notas_cuarto$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para QUINTO
#H0 ---> Los tres turnos de QUINTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_quinto$quinto, notas_quinto$turno)
pairwise.wilcox.test(notas_quinto$quinto, notas_quinto$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para SEXTO
#H0 ---> Los tres turnos de SEXTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_sexto$sexto, notas_sexto$turno)
pairwise.wilcox.test(notas_sexto$sexto, notas_sexto$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

### LENGUA

### A***

In [ ]:
# Creamos un dataframe combinado con los datos de todos los años de curso
datos_completos <- bind_rows(
  mutate(filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "LENGUA"), Año_curso = "PRIMERO"),
  mutate(filter(Victoria_primaria, Año_curso == "SEGUNDO", asignatura == "LENGUA"), Año_curso = "SEGUNDO"),
  mutate(filter(Victoria_primaria, Año_curso == "TERCERO", asignatura == "LENGUA"), Año_curso = "TERCERO"),
  mutate(filter(Victoria_primaria, Año_curso == "CUARTO", asignatura == "LENGUA"), Año_curso = "CUARTO"),
  mutate(filter(Victoria_primaria, Año_curso == "QUINTO", asignatura == "LENGUA"), Año_curso = "QUINTO"),
  mutate(filter(Victoria_primaria, Año_curso == "SEXTO", asignatura == "LENGUA"), Año_curso = "SEXTO")
)

# Filtramos los datos para el turno MAÑANA
datos_mañana <- datos_completos %>%
  filter(turno == "MAÑANA")

# Filtramos los datos para el turno TARDE
datos_tarde <- datos_completos %>%
  filter(turno == "TARDE")

# Filtramos los datos para el turno COMPLETO
datos_completo <- datos_completos %>%
filter(turno == "COMPLETO")

# Creamos un gráfico interactivo
plot <- plot_ly() %>%
  add_boxplot(data = datos_mañana, x = ~Año_curso, y = ~Promedio, name = "MAÑANA", marker = list(color = "blue"), offsetgroup = 1) %>%
  add_boxplot(data = datos_tarde, x = ~Año_curso, y = ~Promedio, name = "TARDE", marker = list(color = "red"), offsetgroup = 2) %>%
  add_boxplot(data = datos_completo, x = ~Año_curso, y = ~Promedio, name = "COMPLETO", marker = list(color = "greem"), offsetgroup = 3) %>%
  layout(
    title = "Comparación de Promedios de Lengua por Año de Curso y Turno",
    xaxis = list(title = "Año de Curso"),
    yaxis = list(title = "Promedio"),
    barmode = "group",
    boxmode = "group"
  )
plot

### B***

In [ ]:
# Analisis para primer grado en LENGUA
datos_mañana1 <- filter(Victoria_primaria, Año_curso == "PRIMERO" & turno == "MAÑANA" & asignatura == "LENGUA")$Promedio
datos_tarde1 <- filter(Victoria_primaria, Año_curso == "PRIMERO" & turno == "TARDE" & asignatura == "LENGUA")$Promedio
datos_completo1 <- filter(Victoria_primaria, Año_curso == "PRIMERO" & turno == "COMPLETO" & asignatura == "LENGUA")$Promedio

# Pasamos los promedios a tipo numerico
primero = as.numeric(c(datos_mañana1, datos_tarde1, datos_completo1))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana1)), rep("TARDE",length(datos_tarde1)), rep("COMPLETO", length(datos_completo1))))
# Armamos el data frame
notas_primero <- data.frame(turno, primero)

# Mostramos la media de cada turno y su cantidad de datos
notas_primero %>% group_by(turno) %>%
  summarise(Media = round(mean(primero),1), datos= n())

# Test de ANOVA
fm1 = aov( primero ~ turno )
#H0 ---> Los tres turnos de PRIMERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm1)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(75, 272, 219), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para segundo grado en LENGUA
datos_mañana2 <- filter(Victoria_primaria, Año_curso == "SEGUNDO" & turno == "MAÑANA" & asignatura == "LENGUA")$Promedio
datos_tarde2 <- filter(Victoria_primaria, Año_curso == "SEGUNDO" & turno == "TARDE" & asignatura == "LENGUA")$Promedio
datos_completo2 <- filter(Victoria_primaria, Año_curso == "SEGUNDO" & turno == "COMPLETO" & asignatura == "LENGUA")$Promedio

# Pasamos los promedios a tipo numerico
segundo = as.numeric(c(datos_mañana2, datos_tarde2, datos_completo2))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana2)), rep("TARDE",length(datos_tarde2)), rep("COMPLETO", length(datos_completo2))))
# Armamos el data frame
notas_segundo <- data.frame(turno, segundo)

# Mostramos la media de cada turno y su cantidad de datos
notas_segundo %>% group_by(turno) %>%
  summarise(Media = round(mean(segundo),1), datos= n())

# Test de ANOVA
fm2 = aov( segundo ~ turno )
#H0 ---> Los tres turnos de SEGUNDO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm2)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(83, 237, 237), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.94
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para tercer grado en LENGUA
datos_mañana3 <- filter(Victoria_primaria, Año_curso == "TERCERO" & turno == "MAÑANA" & asignatura == "LENGUA")$Promedio
datos_tarde3 <- filter(Victoria_primaria, Año_curso == "TERCERO" & turno == "TARDE" & asignatura == "LENGUA")$Promedio
datos_completo3 <- filter(Victoria_primaria, Año_curso == "TERCERO" & turno == "COMPLETO" & asignatura == "LENGUA")$Promedio

# Pasamos los promedios a tipo numerico
tercero = as.numeric(c(datos_mañana3, datos_tarde3, datos_completo3))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana3)), rep("TARDE",length(datos_tarde3)), rep("COMPLETO", length(datos_completo3))))
# Armamos el data frame
notas_tercero <- data.frame(turno, tercero)

# Mostramos la media de cada turno y su cantidad de datos
notas_tercero %>% group_by(turno) %>%
  summarise(Media = round(mean(tercero),1), datos= n())

# Test de ANOVA
fm3 = aov( tercero ~ turno )
#H0 ---> Los tres turnos de TERCERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm3)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(90, 255, 248), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.96
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para cuarto grado en LENGUA
datos_mañana4 <- filter(Victoria_primaria, Año_curso == "CUARTO" & turno == "MAÑANA" & asignatura == "LENGUA")$Promedio
datos_tarde4 <- filter(Victoria_primaria, Año_curso == "CUARTO" & turno == "TARDE" & asignatura == "LENGUA")$Promedio
datos_completo4 <- filter(Victoria_primaria, Año_curso == "CUARTO" & turno == "COMPLETO" & asignatura == "LENGUA")$Promedio

# Pasamos los promedios a tipo numerico
cuarto = as.numeric(c(datos_mañana4, datos_tarde4, datos_completo4))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana4)), rep("TARDE",length(datos_tarde4)), rep("COMPLETO", length(datos_completo4))))
# Armamos el data frame
notas_cuarto <- data.frame(turno, cuarto)

# Mostramos la media de cada turno y su cantidad de datos
notas_cuarto %>% group_by(turno) %>%
  summarise(Media = round(mean(cuarto),1), datos= n())

# Test de ANOVA
fm4 = aov( cuarto ~ turno )
#H0 ---> Los tres turnos de CUARTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm4)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(66, 226, 209), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.88
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para quinto grado en LENGUA
datos_mañana5 <- filter(Victoria_primaria, Año_curso == "QUINTO" & turno == "MAÑANA" & asignatura == "LENGUA")$Promedio
datos_tarde5 <- filter(Victoria_primaria, Año_curso == "QUINTO" & turno == "TARDE" & asignatura == "LENGUA")$Promedio
datos_completo5 <- filter(Victoria_primaria, Año_curso == "QUINTO" & turno == "COMPLETO" & asignatura == "LENGUA")$Promedio

# Pasamos los promedios a tipo numerico
quinto = as.numeric(c(datos_mañana5, datos_tarde5, datos_completo5))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana5)), rep("TARDE",length(datos_tarde5)), rep("COMPLETO", length(datos_completo5))))
# Armamos el data frame
notas_quinto <- data.frame(turno, quinto)

# Mostramos la media de cada turno y su cantidad de datos
notas_quinto %>% group_by(turno) %>%
  summarise(Media = round(mean(quinto),1), datos= n())

# Test de ANOVA
fm5 = aov( quinto ~ turno )
#H0 ---> Los tres turnos de QUINTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm5)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(76, 272, 216), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

In [ ]:
# Analisis para sexto grado en LENGUA
datos_mañana6 <- filter(Victoria_primaria, Año_curso == "SEXTO" & turno == "MAÑANA" & asignatura == "LENGUA")$Promedio
datos_tarde6 <- filter(Victoria_primaria, Año_curso == "SEXTO" & turno == "TARDE" & asignatura == "LENGUA")$Promedio
datos_completo6 <- filter(Victoria_primaria, Año_curso == "SEXTO" & turno == "COMPLETO" & asignatura == "LENGUA")$Promedio

# Pasamos los promedios a tipo numerico
sexto = as.numeric(c(datos_mañana6, datos_tarde6, datos_completo6))
# Segun la cantidad de datos en cada turno, se repite su nombre
turno <- as.factor(c(rep("MAÑANA", length(datos_mañana6)), rep("TARDE",length(datos_tarde6)), rep("COMPLETO", length(datos_completo6))))
# Armamos el data frame
notas_sexto <- data.frame(turno, sexto)

# Mostramos la media de cada turno y su cantidad de datos
notas_sexto %>% group_by(turno) %>%
  summarise(Media = round(mean(sexto),1), datos= n())

# Test de ANOVA
fm6 = aov( sexto ~ turno )
#H0 ---> Los tres turnos de SEXTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
summary(fm6)
# Con un nivel de significancia de 0.05 podemos decir que hay suficiente evidencia para rechazar H0
#en favor de H1 y afirmar que al menos un conjunto de datos tiene una media distinta que los demas

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(85, 279, 219), f = 0.25, sig.level = 0.05)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.95
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

### C***

In [ ]:
# Despues de hacer el test de ANOVA para la variable turno de todos los grados de primaria, los resultados
#se repitieron en todos los cursos, al menos una media de un conjunto de los datos es distinta a las demas
#en algunos casos las diferencias eran muy significativas y en otras muy leves.

### D***

In [ ]:
# Test HSD de Tukey para PRIMERO
TukeyHSD(fm1, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0000018
# TARDE-COMPLETO -> 0.0000497

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(75, 272, 219), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para SEGUNDO
TukeyHSD(fm2, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0
# TARDE-COMPLETO -> 0.0

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(83, 237, 237), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.94
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para TERCERO
TukeyHSD(fm3, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0000007
# TARDE-COMPLETO -> 0.0000001

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(90, 255, 248), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.96
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para CUARTO
TukeyHSD(fm4, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0273083
# TARDE-COMPLETO -> 0.0259945

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(66, 226, 209), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.88
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para QUINTO
TukeyHSD(fm5, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0000829
# TARDE-COMPLETO -> 0.0010366

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(76, 272, 216), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.92
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

print("----------------------------------------------------------------------------------------")
# Test HSD de Tukey para SEXTO
TukeyHSD(fm6, "turno", conf.level = .95)
# MAÑANA-COMPLETO -> 0.0000042
# TARDE-COMPLETO -> 0.0

# Potencia del test
potencia <- pwr.anova.test(k = 3, n = c(85, 279, 219), f = 0.25)$power
potencia
# La potencia del test para cada conjunto de datos es muy alta
# Turno completo ---> 0.95
# Turno mañana ---> 0.99
# Turno tarde ---> 0.99

### E***

In [ ]:
# Prueba de normalidad para PRIMERO
shapiro.test(notas_primero$primero[notas_primero$turno == "MAÑANA"])
shapiro.test(notas_primero$primero[notas_primero$turno == "TARDE"])
shapiro.test(notas_primero$primero[notas_primero$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para SEGUNDO
shapiro.test(notas_segundo$segundo[notas_segundo$turno == "MAÑANA"])
shapiro.test(notas_segundo$segundo[notas_segundo$turno == "TARDE"])
shapiro.test(notas_segundo$segundo[notas_segundo$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para TERCERO
shapiro.test(notas_tercero$tercero[notas_tercero$turno == "MAÑANA"])
shapiro.test(notas_tercero$tercero[notas_tercero$turno == "TARDE"])
shapiro.test(notas_tercero$tercero[notas_tercero$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para CUARTO
shapiro.test(notas_cuarto$cuarto[notas_cuarto$turno == "MAÑANA"])
shapiro.test(notas_cuarto$cuarto[notas_cuarto$turno == "TARDE"])
shapiro.test(notas_cuarto$cuarto[notas_cuarto$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para QUINTO
shapiro.test(notas_quinto$quinto[notas_quinto$turno == "MAÑANA"])
shapiro.test(notas_quinto$quinto[notas_quinto$turno == "TARDE"])
shapiro.test(notas_quinto$quinto[notas_quinto$turno == "COMPLETO"])

print("----------------------------------------------------------------------------------------")

# Prueba de normalidad para SEXTO
shapiro.test(notas_sexto$sexto[notas_sexto$turno == "MAÑANA"])
shapiro.test(notas_sexto$sexto[notas_sexto$turno == "TARDE"])
shapiro.test(notas_sexto$sexto[notas_sexto$turno == "COMPLETO"])

# Ningun conjunto de datos cumple con los supuestos para realizar ANOVA, asi que el análisis realizado 
#anteriormente no es confiable

### F***

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

print("----------------------------------------------------------------------------------------")

# Test no parametrico para PRIMERO
#H0 ---> Los tres turnos de PRIMERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_primero$primero, notas_primero$turno)
pairwise.wilcox.test(notas_primero$primero, notas_primero$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para SEGUNDO
#H0 ---> Los tres turnos de SEGUNDO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_segundo$segundo, notas_segundo$turno)
pairwise.wilcox.test(notas_segundo$segundo, notas_segundo$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para TERCERO
#H0 ---> Los tres turnos de TERCERO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_tercero$tercero, notas_tercero$turno)
pairwise.wilcox.test(notas_tercero$tercero, notas_tercero$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para CUARTO
#H0 ---> Los tres turnos de CUARTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_cuarto$cuarto, notas_cuarto$turno)
pairwise.wilcox.test(notas_cuarto$cuarto, notas_cuarto$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para QUINTO
#H0 ---> Los tres turnos de QUINTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_quinto$quinto, notas_quinto$turno)
pairwise.wilcox.test(notas_quinto$quinto, notas_quinto$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

print("----------------------------------------------------------------------------------------")

# Test no parametrico para SEXTO
#H0 ---> Los tres turnos de SEXTO tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(notas_sexto$sexto, notas_sexto$turno)
pairwise.wilcox.test(notas_sexto$sexto, notas_sexto$turno)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

### Actividad 4

### Muestras independientes

***El rendimiento de los alumnos a lo largo del año se ve afectado por situaciones que afectan directamente en el rendimiento de los alumnos, entonces se puede tener un muy buen rendimiento en el primer trimestre, pero en el segundo verse afectado por situaciones externas lo cual logra disminuir su rendimiento, pero en el tercer trimestre verse fortalecido y mejorar su rendimiento equiparandolo con lo logrado en el primer trimestre. Por eso consideramos que las notas conseguidas por cada estudiante en los distintos años son muestras independientes.***

### A***

In [ ]:
# Filtramos los datos para el primer trimestre
datos_primer_tri <- filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasPrimerT

# Filtramos los datos para el segundo trimestre
datos_segundo_tri <- filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasSegundoT

# Filtramos los datos para el tercer trimestre
datos_tercer_tri <- filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri)
)

# Convertimos Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Primero",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    barmode = "group",
    boxmode = "group"
  )
plot

### B***

In [ ]:
# Al querer hacer una comparación para los tres trimestres en primer grado de Matemática,deberemos
#usar el test de ANOVA de un factor en caso de que cumpla los supuestos y sino con el test de 
#Kruskal

In [ ]:
shapiro.test(datos_primer_tri)
print("------------------------------------------------------------------------")
shapiro.test(datos_segundo_tri)
print("------------------------------------------------------------------------")
shapiro.test(datos_tercer_tri)

# Como no cumple con los supuestos de tener una distribucion normal, deberemos usar el test de Kruskal

In [ ]:
# Valor critico
qchisq(0.05, 3-1, lower.tail = F)

print("----------------------------------------------------------------------------------------")

# Test no parametrico para PRIMERO en Matemática
#H0 ---> Los tres trimestres de PRIMERO en Matemática tienen la misma media
#H1 ---> Hay un grupo de datos que tiene una media distinta
kruskal.test(datos_completos$Notas, datos_completos$Trimestre)
# Hay evidencia suficiente para rechazar H0 en favor de H1 y decir que existen diferencias entre los turnos

### C***

In [ ]:
pairwise.wilcox.test(datos_completos$Notas, datos_completos$Trimestre)
# Primer Trimestre tiene grandes diferencias con los demas trimestres. 
#Pero Segundo y Tercer Trimestre tienen una diferencia muy baja

### Muestras pareadas

### A***

In [ ]:
# Filtramos los datos para el primer trimestre
datos_primer_tri <- filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasPrimerT

# Filtramos los datos para el segundo trimestre
datos_segundo_tri <- filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasSegundoT

# Filtramos los datos para el tercer trimestre
datos_tercer_tri <- filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasTercerT

# Unimos los datos filtrados a un data frame
datos_completos <- data.frame(
  Trimestre = rep(c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"), each = length(datos_primer_tri)),
  Notas = c(datos_primer_tri, datos_segundo_tri, datos_tercer_tri)
)

# Convertimos Trimestre a factor
datos_completos$Trimestre <- factor(datos_completos$Trimestre, levels = c("Primer Trimestre", "Segundo Trimestre", "Tercer Trimestre"))

# Creamos un gráfico interactivo
plot <- plot_ly() %>%
  add_boxplot(data = datos_completos, x = ~Trimestre, y = ~Notas, color = ~Trimestre, colors = c("blue", "red", "green"), name = "Matemáticas", offsetgroup = 1) %>%
 layout(
    title = "Comparación de notas por trimestre de Matemáticas en Primero",
    xaxis = list(title = "Trimestre"),
    yaxis = list(title = "Notas"),
    barmode = "group",
    boxmode = "group"
  )
plot

### B***

In [ ]:
# Calculamos el tamaño de los alumnos de primer grado en Matemática para utilizar posteriormente
tamanio = length(datos_primer_tri)

# Armamos el data frame con lo datos
notas_primero <- data.frame(
 sujeto = 1:tamanio,
 notas_primer_tri = filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasPrimerT,
 notas_segundo_tri = filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasSegundoT,
 notas_tercer_tri = filter(Victoria_primaria, Año_curso == "PRIMERO", asignatura == "MATEMATICA")$NotasTercerT
)

# Esto sirve para armar los datos de manera larga ¿QUE?
datos_largo <- melt(notas_primero, id.vars= "sujeto", variable.name= "Notas", value.name= "Puntuacion")

# Realizar el ANOVA de medidas pareadas
modelo_anova <- aov(Puntuacion ~ Notas + Error(sujeto/Notas), data = datos_largo)

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calculamos normalidad de los residuos y que las varianzas sean constantes
print("Test de Normalidad:")
shapiro.test(modelo_lm$residuals)
print("Test de homocedasticidad: ")
bartlett.test(modelo_lm$residuals ~ datos_largo$Notas)
# Los residuos no son normales y las varianzas no son constantes. Entonces se utilizara el test de Friedman

In [ ]:
# Armamos el data frame con lo datos
notas= as.numeric(c(datos_primer_tri,datos_segundo_tri,datos_tercer_tri))
trimestres <- as.factor(c(rep("Primer Trimestre",length(datos_primer_tri)), rep("Segundo Trimestre",length(datos_segundo_tri)),rep("Tercer Trimestre",length(datos_tercer_tri))))
datos <- data.frame(Alumnos=rep(1:tamanio,3), notas, trimestres)
head(datos)

# Calculamos medias de cada grupo
by(data = datos$notas, INDICES = datos$trimestres, FUN = median)

# Test para encontrar diferencias significativas entre los grupos
friedman.test(notas ~ trimestres | Alumnos, data=datos)
# Se puede notar una gran diferencia entre las notas entre los diferentes trimestres 

# Grafico de coordenadas paralelas
datos_grafica <- datos_largo %>%
  filter(sujeto <= 10, Notas %in% c("notas_primer_tri", "notas_segundo_tri","notas_tercer_tri"))
ggplot(datos_grafica, aes(x = Notas, y = Puntuacion, group = sujeto, color = factor(sujeto))) +
  geom_line() +
  geom_point() +
  labs(title = "Evolución de algunas de las notas trimestrales a lo largo del año 2023
    de lo alumnos de Primer Grado en Matematica",
       x = "Trimestre",
       y = "Nota") +
  theme_minimal()

# Potencia del resultado 
friedman_anova <- pwr.anova.test(k = 3, n = tamanio, f = 0.25, sig.level = 0.05)$power
print("Potencia del test:")
friedman_anova

### C***

In [ ]:
# Para saber entre que conjuntos de datos estan las diferencias
pairwise.wilcox.test(datos$notas, datos$trimestres, paired = TRUE)
# Las diferencias son significativas entre los tres trimestres

### ¿Que otras variables podemos utilizar para usar estos test?

### Anexo

In [ ]:
# Calculamos las medidas de centralidad
media_ciclo_basico <- mean(ciclo_basico)
mediana_ciclo_basico <- median(ciclo_basico)
media_ciclo_orientado <- mean(ciclo_orientado)
mediana_ciclo_orientado <- median(ciclo_orientado)

# Boxplot ciclo básico
boxplot(ciclo_basico,
        main = "Boxplot para los Promedios en Ciclo Basico",
        whisklty = 1,       # Tipo línea bigotes
        whisklwd = 3,       # Ancho línea bigotes
        whiskcol = "black",   # Color bigotes
        staplelty = 1,      # Tipo línea barras
        staplelwd = 3,      # Ancho línea barras
        staplecol = "black",  # Color barras
        col = "green",     # Color caja
        sub = "Fuente de elaboración propia (Gráfico N°1)",
        col.sub = "black")
abline(h = media_ciclo_basico, col = "blue", lty = , lwd= 2)
legend("topleft", legend = paste("Media:", round(media_ciclo_basico, 2)), col = "blue", lty = 1, lwd= 2, pch = NA)
legend("bottomright", legend = paste("Mediana:", round(mediana_ciclo_basico, 2)), col = "black", lty = 1, lwd= 2, pch = NA) 

# Boxplot ciclo orientado
boxplot(ciclo_orientado,
        main = "Boxplot para los Promedios en Ciclo Orientado",
        whisklty = 1,       # Tipo línea bigotes
        whisklwd = 3,       # Ancho línea bigotes
        whiskcol = "black",   # Color bigotes
        staplelty = 1,      # Tipo línea barras
        staplelwd = 3,      # Ancho línea barras
        staplecol = "black",  # Color barras
        col = "skyblue",     # Color caja
        sub = "Fuente de elaboración propia (Gráfico N°2)",
        col.sub = "black")
abline(h = media_ciclo_orientado, col = "red", lty = 1, lwd= 2)
legend("topleft", legend = paste("Media:", round(media_ciclo_orientado, 2)), col = "red", lty = 1, lwd= 2, pch = NA)
legend("bottomright", legend = paste("Mediana:", round(mediana_ciclo_orientado, 2)), col = "black", lty = 1, lwd= 2, pch = NA) 

In [ ]:
# crear nuevas variables
# cantidad de materias aprobadas
# cantidad de materias desaprobadas
# cantidad total de materias de cada alumno
# edad de cada alumno, especificar que no se pudo hacer xq no tiene una fuente confiable con los dni por encima de 90.000.000
# cambiar variable es_privada
#
#
#
#
# ver si las bajas notas es por inasistencia o que carajos